In [ ]:
pip install folium mapclassify
pip install ipywidgets
pip install leafmap
pip install lonboard palettable sidecar

In [10]:
pip install lonboard palettable sidecar

   ---------------------------------------- 0.0/884.1 kB ? eta -:--:--
   ---- ----------------------------------- 102.4/884.1 kB 3.0 MB/s eta 0:00:01
   -------------------------- ------------- 593.9/884.1 kB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 884.1/884.1 kB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/332.3 kB ? eta -:--:--
   --------------------------------------- 332.3/332.3 kB 10.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   -------- ------------------------------- 0.6/2.7 MB 12.2 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.7 MB 13.3 MB/s eta 0:00:01
   ----------------------- ---------------- 1.6/2.7 MB 11.1 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.7 MB 11.1 MB/s eta 0:00:01
   ------------------------------------- -- 2.5/2.7 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 10.2 MB/s eta 0:00:00
   ----

In [5]:
pip install leafmap

     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ---------------------------------------- 56.0/56.0 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/534.4 kB ? eta -:--:--
   ---------------------- ----------------- 307.2/534.4 kB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 534.4/534.4 kB 8.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/220.7 kB ? eta -:--:--
   ---------------------------------------- 220.7/220.7 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------- ----------------------- 0.5/1.2 MB 10.7 MB/s eta 0:00:01
   ---------------------------------- ----- 1.1/1.2 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 9.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.4 MB 11.4 MB/s eta 0:00:01
   --- ------------

In [12]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import zipfile
import requests
from datetime import datetime, timedelta
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import leafmap
import shapely
from palettable.colorbrewer.diverging import BrBG_10
from sidecar import Sidecar

from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
from sqlalchemy import create_engine
import duckdb



## LOAD SMOKE SHAPEFILE SCRIPT

In [6]:
base_dir = r"E:\Capstone\hms_smoke_2023_to_2024"

smoke = []

for folder in tqdm(sorted(os.listdir(base_dir))):
    folder_path = os.path.join(base_dir, folder)

    if os.path.isdir(folder_path):
        # Extract the date string from folder name (assumes format: hms_smoke_YYYYMMDD)
        try:
            date_str = folder.split("_")[-1]  # "20230101"
            date = pd.to_datetime(date_str, format="%Y%m%d")
        except Exception as e:
            print(f"Skipping folder {folder} due to date parsing error: {e}")
            continue

        # Look for .shp file inside the folder
        for file in os.listdir(folder_path):
            if file.endswith(".shp"):
                shp_path = os.path.join(folder_path, file)
                try:
                    gdf = gpd.read_file(shp_path)
                    gdf["date"] = date  # assign parsed date
                    smoke.append(gdf)
                except Exception as e:
                    print(f"Failed to read {shp_path}: {e}")
                break  # Only one shapefile per folder

# Combine all into one GeoDataFrame
if smoke:
    all_smoke_gdf = gpd.GeoDataFrame(pd.concat(smoke, ignore_index=True))
    all_smoke_gdf.head()
else:
    print("No shapefiles loaded.")

100%|███████████████████████████████████████████████████████████████████████████████| 731/731 [00:01<00:00, 467.22it/s]


## LOAD FIRE SHAPEFILE SCRIPT

In [ ]:
import os
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

# Set the path to your fire shapefiles folder
base_dir = r"E:\Capstone\hms_shapefile_fire_2023_to_2024"

# Empty list to store daily GeoDataFrames
fire = []

# Loop through each subfolder (one per day)
for folder in tqdm(sorted(os.listdir(base_dir))):
    folder_path = os.path.join(base_dir, folder)

    if os.path.isdir(folder_path):
        try:
            # Extract date from folder name: e.g., 'hms_fire_20230101' -> '20230101'
            date_str = folder.split("_")[-1]
            date = pd.to_datetime(date_str, format="%Y%m%d")
        except Exception as e:
            print(f"Skipping folder {folder}: couldn't parse date - {e}")
            continue

        # Look for the .shp file inside the folder
        for file in os.listdir(folder_path):
            if file.endswith(".shp"):
                shp_path = os.path.join(folder_path, file)
                try:
                    gdf = gpd.read_file(shp_path)
                    gdf["date"] = date  # Add a date column
                    fire.append(gdf)
                except Exception as e:
                    print(f"Failed to read {shp_path}: {e}")
                break  # Only one shapefile per folder

# Combine all GeoDataFrames into one
if fire:
    all_fire_gdf = gpd.GeoDataFrame(pd.concat(fire, ignore_index=True))
    all_fire_gdf.head()
else:
    print("No fire shapefiles were successfully loaded.")


100%|████████████████████████████████████████████████████████████████████████████████| 731/731 [02:22<00:00,  5.12it/s]


In [8]:
m = leafmap.Map(center=[0,0], zoom=2)

In [9]:
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

# DUCKDB setup

In [ ]:
con = duckdb.connect()

In [ ]:
con.query('INSTALL spatial')
con.query('LOAD spatial')